In [ ]:
'''
A Multilayer Perceptron implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

In [3]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 10 # 1st layer number of features #256 originally
n_hidden_2 = 10 # 2nd layer number of features #256 origianlly
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [5]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [6]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [7]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))
    
    # Backup params
    params = []
    for v in tf.trainable_variables():
        params.append(v.eval(sess))

Epoch: 0001 cost= 8.476727576
Epoch: 0002 cost= 2.769662222
Epoch: 0003 cost= 2.489101746
Epoch: 0004 cost= 2.375026234
Epoch: 0005 cost= 2.299229826
Epoch: 0006 cost= 2.199927914
Epoch: 0007 cost= 1.861813587
Epoch: 0008 cost= 1.545338369
Epoch: 0009 cost= 1.408154682
Epoch: 0010 cost= 1.339001037
Epoch: 0011 cost= 1.287556763
Epoch: 0012 cost= 1.238149140
Epoch: 0013 cost= 1.190676829
Epoch: 0014 cost= 1.135632247
Epoch: 0015 cost= 1.075815447
Optimization Finished!
Accuracy: 0.5972


In [8]:
# JSONify data
import numpy as np
import json

def SimpleEncode(ndarray):
    return json.dumps(ndarray.tolist())
#def SimpleDecode(jsonDump):
#    return np.array(json.loads(jsonDump))

param_names = ["W1", "W1", "Wout", "b1", "b2", "bout"]

i = 0
for param_name in param_names:
    print(str(i) + ". Param \"" + param_name + "\"")
    json_string = SimpleEncode(params[i].astype(np.float32))
    print(params[i].shape)
    #%timeit SimpleDecode(json_string)
    with open(param_name + '.json', 'w') as outfile:
        outfile.write(json_string)
        outfile.close
    print(params[i][0])
    i = i + 1
    
# Test images
json_string = SimpleEncode(mnist.test.images[:100].astype(np.float32))
with open('test_images.json', 'w') as outfile:
    outfile.write(json_string)
    outfile.close
# Test labels
json_string = SimpleEncode(mnist.test.labels[:100].astype(np.float32))
with open('test_labels.json', 'w') as outfile:
    outfile.write(json_string)
    outfile.close

0. Param "W1"
(784, 10)
[-0.16735978  0.39669412 -1.17989802  0.35566193 -0.39502484 -1.34435642
 -1.97181439  0.88933438  0.61063141  0.88226837]
1. Param "W1"
(10, 10)
[-0.53594834  0.85999078  1.00576627 -0.58123481  0.16745444  0.26550797
 -1.75466371 -0.0247819  -2.20835304  0.81838751]
2. Param "Wout"
(10, 10)
[-1.19202924 -0.00310809  0.09103666  0.48054278  0.18368714  0.44561648
  2.37772608 -0.48481271  0.0131591  -0.79109699]
3. Param "b1"
(10,)
-1.34601
4. Param "b2"
(10,)
-0.538608
5. Param "bout"
(10,)
1.42716


In [12]:
# Store layers weight & bias

x = tf.placeholder("float", [None, n_input])
weights = {
    'h1': tf.Variable(params[0]),
    'h2': tf.Variable(params[1]),
    'out': tf.Variable(params[2]),
}
biases = {
    'b1': tf.Variable(params[3]),
    'b2': tf.Variable(params[4]),
    'out': tf.Variable(params[5])
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Produce outputs
    #or image in mnist.test.images[:100]:
    #mnist.test.images[:100]
    batch_x, _ = mnist.train.next_batch(batch_size)
    result = sess.run([pred], feed_dict={x: batch_x})
    
print(result)

[array([[  5.71163595e-01,  -7.95173883e-01,   1.56279278e+00,
          2.82917404e+00,  -4.14929539e-02,   2.45012045e+00,
         -8.95569026e-01,  -2.07581568e+00,   2.56186152e+00,
          1.79130077e-01],
       [ -1.15906410e+01,   9.16491508e+00,   1.26727543e+01,
         -2.28081512e+01,   1.32926025e+01,   9.52019882e+00,
          2.05422440e+01,  -4.50685453e+00,   4.70522785e+00,
          7.71781254e+00],
       [  5.71163595e-01,  -7.95173883e-01,   1.56279278e+00,
          2.82917404e+00,  -4.14929539e-02,   2.45012045e+00,
         -8.95569026e-01,  -2.07581568e+00,   2.56186152e+00,
          1.79130077e-01],
       [ -3.75236559e+00,   2.71715331e+00,   1.77469420e+00,
         -7.48429298e-01,   1.68936062e+00,   3.33380222e+00,
          2.77287984e+00,  -6.21087551e-01,   3.74516845e+00,
          8.14125061e-01],
       [ -8.15742397e+00,  -4.37158203e+00,  -9.89613056e-01,
         -3.69827271e-01,   1.43350542e+00,  -1.79088688e+00,
         -4.09171772e+0